# План домашней работы
1. Реализовать алгоритмы **item2item**, **ALS**, **IALS** (2 балл за каждый)
2. Посчитать метрику предсказаний **MRR@100** выбрасывая случайный лайк пользователя (2 балла)

Будем решать задачу предсказания: на 4/5 пользователей учимся, на 1/5 выбрасываем случайный лайк и пытаемся предсказать его беря топ 100 наших лучших предсказаний для этого пользователя.

MRR@100 будет равно $1/(p+1)$, где $p$ - позиция на которой оказался выброшенный лайк в нашем ранжировании и 0 если в топ 100 его не было.

3. Подобрать параметры алгоритмов для максимизации MRR@100 (1 балл)
4. Сравнить похожести айтемов получаюшиеся для item2item, ALS, IALS (1 балл)

Замерить насколько получаются похожими топы похожестей. Так же рекомендуется взять 5-топовых (или любимых) статей и посмотреть на похожести которые получаются для них в разных алгоритмах.

### Обозначения:
1. item_id - уникальный id айтема (статьи).
2. user_id - уникальный id пользователя.
3. source_id - уникальный id автора. Если у двух айтемов совпадают source_id, то это статьи одного автора.
4. Название айтема - это название статьи.
5. В датасете хранится user_id и список айтемов item_ids, с которыми пользователь положительно провзаимодействовал.

### Ссылки на датасет Дзена:

1. Датасет: https://disk.yandex.ru/d/uUx1MMsZUR87Sw
2. Названия айтемов: https://disk.yandex.ru/d/_ZMXsmki-OtLJA
3. Линки между item_id и source_id: https://disk.yandex.ru/d/GCryohhLbYPFoA

# Разбираемся с датасетом Дзена

In [74]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm.notebook import tqdm
from implicit.nearest_neighbours import ItemItemRecommender

In [2]:
all_names = pd.read_json("data/item_id_to_name.json", lines=False)
item_links = pd.read_json("data/item_id_to_source_id.json", lines=False)
dataset = pd.read_json("data/dataset_zen.json", lines=False)

In [3]:
item_links

,id,source
0,94962,2919814402697966089
1,3972,3263022753228392991
2,94644,-3857390427602554682
3,82518,-9036908390349249792
4,53264,3353856219169766284
...,...,...
104498,36769,3818746211375738614
104499,9190,4975535765688979937
104500,52731,3720366796439288909
104501,72660,-7860042973720636310


In [4]:
dataset

,user_id,item_ids
0,993675863667353526,"[15267, 61075, 81203, 17066, 25471, 88427, 638..."
1,4250619547882954185,"[4555, 94644, 84972, 17774, 94962, 78217, 2485..."
2,3847785305345691076,"[1898, 26703, 16525, 86939, 55017, 31069, 4035..."
3,1785181112918558233,"[75601, 102458, 28716, 100694, 5757, 47104, 60..."
4,5078748097863903181,"[72260, 40825, 2615, 42549, 379, 100818, 56827..."
...,...,...
75905,4954138831959898373,"[11881, 55520, 63054, 48015, 66952, 103830, 21..."
75906,4967793435819938014,"[74697, 11830, 63858, 87245, 41956, 62089, 686..."
75907,7137764184903122777,"[10353, 1775, 103680, 29704, 9782, 13295, 9975..."
75908,2624987805086334956,"[24324, 18854, 73319, 66641, 64078, 97387, 426..."


In [5]:
total_interactions_count = dataset.item_ids.map(len).sum()
user_coo = np.zeros(total_interactions_count, dtype=np.int64)
item_coo = np.zeros(total_interactions_count, dtype=np.int64)
pos = 0

for user_id, item_ids in enumerate(dataset.item_ids):
    user_coo[pos:pos+len(item_ids)] = user_id
    item_coo[pos:pos+len(item_ids)] = item_ids
    pos += len(item_ids)
shape = (max(user_coo) + 1, max(item_coo) + 1)
user_item_matrix = sp.coo_matrix((np.ones(len(user_coo)), (user_coo, item_coo)), shape=shape)
user_item_matrix = user_item_matrix.tocsr()
sp.save_npz("data_train.npz", user_item_matrix)
# Cleanup memory. Later you need just data_train.npz
del user_coo
del item_coo
del dataset

In [6]:
user_item_matrix = sp.load_npz("data_train.npz")

In [7]:
user_item_matrix

<75910x104503 sparse matrix of type '<class 'numpy.float64'>'
	with 5792423 stored elements in Compressed Sparse Row format>

In [8]:
item_weights = np.array(user_item_matrix.tocsc().sum(0))[0]
top_to_bottom_order = np.argsort(-item_weights)
item_mapping = np.empty(top_to_bottom_order.shape, dtype=int)
item_mapping[top_to_bottom_order] = np.arange(len(top_to_bottom_order))
total_item_count = (item_weights > 0).sum()
total_user_count = user_item_matrix.shape[0]

def build_dataset(user_item_matrix, item_pct, user_pct):
    user_count, item_count = int(total_user_count * user_pct), int(total_item_count * item_pct)
    item_ids = top_to_bottom_order[:item_count]
    user_ids = np.random.choice(np.arange(user_item_matrix.shape[0]), size=user_count, replace=False)
    train = user_item_matrix[user_ids]
    train = train[:, item_ids]
    return train

In [9]:
small_dataset = build_dataset(user_item_matrix, 0.05, 0.05)

In [10]:
small_dataset

<3795x5019 sparse matrix of type '<class 'numpy.float64'>'
	with 135873 stored elements in Compressed Sparse Row format>

## 1. Вспомогательные методы

In [11]:
def train_test_split(data, percentage=0.8):
    test_size = int(data.shape[0] * (1.0 - percentage))

    test_user_ids = []
    for i in range(data.shape[0]):
        if data[i].nnz > 0:
            test_user_ids.append(i)
        if len(test_user_ids) >= test_size:
            break

    train_user_ids = np.array(list(set(np.arange(data.shape[0])) - set(test_user_ids)))
    test_user_ids = np.array(test_user_ids)
    return data[train_user_ids], data[test_user_ids]

In [12]:
def get_mrr(actual, predicted):
    result = 0
    for i, item_id in enumerate(actual):
        pos = np.where(predicted[i] == item_id)[0]
        if pos.any():
            result += 1 / (pos.item() + 1)
    return result / actual.shape[0]

In [13]:
def drop_likes(data):
    rows, cols, values = sp.find(data.T)
    y = []
    for u in range(data.shape[0]):
        rand_id = np.random.choice(np.where(cols == u)[0], 1)
        values[rand_id] = 0
        y.append(rows[rand_id].item())
    idx = np.where(values != 0)[0]
    return sp.coo_matrix((values[idx], (cols[idx], rows[idx])), shape=data.shape).tocsr(), np.array(y)

In [94]:
train, valid = train_test_split(small_dataset, 0.8)
test, y = drop_likes(valid)

## 2. Реализация алгоритмов

### 2.1 item2item

Референсы решения:
1. https://colab.research.google.com/drive/1Kpqkeqonb0rVMCggKKEGQFsszfXdY7tq#scrollTo=SpjOat_l5DXI - семинар
2. https://github.com/nzhinusoftcm/review-on-collaborative-filtering/blob/master/3.Item-basedCollaborativeFiltering.ipynb - англоязычный аналог

In [19]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt

In [20]:
item_similarities = cosine_similarity(train.T)
item_similarities.shape

(5019, 5019)

In [21]:
# get neighbors by their neighbors in decreasing order of similarities
neighbors = np.flip(np.argsort(item_similarities), axis=1)

# sort similarities in decreasing order
similarities = np.flip(np.sort(item_similarities), axis=1)

In [22]:
neighbors[4352]

array([4352, 4625, 4853, ..., 3289, 3290, 2509])

In [23]:
def candidate_items(data, neighbors, userid, k=-1):
    _, user_item_ids, _ = sp.find(data[userid])
    user_item_ids = set(user_item_ids)
    candidates = set()
    for item in user_item_ids:
        candidates.update(set(neighbors[item][:k]))
    candidates -= user_item_ids
    return np.array(list(user_item_ids)), np.array(list(candidates))

In [24]:
user_item_ids, u_candidates = candidate_items(test, neighbors, 0, 10)

In [25]:
def similarity_with_user_items(item_id, user_item_ids, similarities, neighbors):
    w = 0
    for i_id in user_item_ids:
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        # your code here
        w += similarities[i_id][np.where(neighbors[i_id] == item_id)[0][0]]
    return w

In [26]:
def rank_candidates(candidates, user_item_ids, similarities, neighbors):
    # list of candidate items mapped to their corresponding similarities to user_item_ids
    sims = [similarity_with_user_items(c, user_item_ids, similarities, neighbors) for c in candidates]
    # candidates = iencoder.inverse_transform(candidates)
    mapping = list(zip(candidates, sims))

    ranked_candidates = sorted(mapping, key=lambda couple:couple[1], reverse=True)
    return ranked_candidates

In [27]:
def topn_recommendation(data, userid, similarities, neighbors, k=-1, N=100):
    # find candidate items
    user_item_ids, candidates = candidate_items(data, neighbors, userid, k)

    # rank candidate items according to their similarities with user_item_ids
    ranked_candidates = rank_candidates(candidates, user_item_ids, similarities, neighbors)

    # get the first N row of ranked_candidates to build the top N recommendation list
    topn = [i[0] for i in ranked_candidates[:N]]
    return topn

In [28]:
def predict_top100(data, similarities, neighbors):
    predictions = []
    for u in tqdm(range(data.shape[0])):
        predictions.append(topn_recommendation(data, u, similarities, neighbors, k=-1, N=100))
    return predictions

In [29]:
i2i_preds = predict_top100(test, similarities, neighbors)

  0%|          | 0/758 [00:00<?, ?it/s]

In [30]:
def get_mrr(actual, predicted):
    result = 0
    for i, item_id in enumerate(actual):
        pos = np.where(predicted[i] == item_id)[0]
        if pos.any():
            result += 1 / (pos.item() + 1)
    return result / actual.shape[0]

In [31]:
get_mrr(y, i2i_preds)

0.017973430038211523

### 2.2 ALS

In [79]:
def calc_oposite_vectors(Y, A, lam=1e-4):
    B = Y.T.dot(Y) + lam * sp.eye(Y.shape[1]) # (k x k)
    return sp.linalg.inv(B).dot(Y.T).dot(A.T).T

In [81]:
def frobenius_norm(data, X, Y):
    return (data - X.dot(Y.T)).power(2).sum()

In [83]:
k = 100

In [84]:
X, Y = sp.random(train.shape[0], k, density=0.25), sp.random(train.shape[1], k, density=0.25)
frobenius_norms = [frobenius_norm(train, X, Y)]
for i in tqdm(range(20)):
    X = calc_oposite_vectors(Y, train)
    frobenius_norms.append(frobenius_norm(train, X, Y))
    Y = calc_oposite_vectors(X, train.T)
    frobenius_norms.append(frobenius_norm(train, X, Y))

  0%|          | 0/20 [00:00<?, ?it/s]

/Users/kr.sivakov/anaconda3/envs/myenv/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/Users/kr.sivakov/anaconda3/envs/myenv/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:285: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


In [85]:
test_user_embs = calc_oposite_vectors(Y, test)
test_als = test_user_embs.dot(Y.T)
als_preds = np.flip(np.argsort(test_als.toarray()), axis=1)

### 2.3 IALS

In [73]:
def get_ials_recommends(recommender, data):
    result = []
    for user_id in tqdm(range(data.shape[0])):
        result.append(recommender.recommend(user_id, data[user_id], 100)[0])
    return np.array(result)

In [75]:
recommender = ItemItemRecommender()
recommender.fit(train)
ials_preds = get_ials_recommends(recommender, test)

  0%|          | 0/5019 [00:00<?, ?it/s]

  0%|          | 0/758 [00:00<?, ?it/s]

/var/folders/qg/qf0y_9l119xdw7z_0pj_zsrc0000gn/T/ipykernel_10447/1391948487.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(result)


In [76]:
ials_preds[0]

array([  18,    6,   39,    3,   58,   12,  305,    2,  141,   60,   20,
         52,   41,   22,   26,   35,   17,  207,   95,  774,  325,  319,
        111,   73,    7, 2487, 1341, 1180,  786,  492,  273,  121,  107,
         66, 1313,  856,  746,  255,   62, 1346,  176,  164,   89,   86,
         81,   27, 4402, 1967, 1044, 1021,  388,  345,  205,  188,   97,
         92,   59, 2385, 2003, 1748, 1382, 1365, 1170,  784,  630,  487,
        476,  469,  387,  303,  295,  289,  259,  250,  124,   25,   14,
       3206, 2509, 2304, 2273, 2042, 1642, 1500, 1288, 1269, 1193,  935,
        896,  799,  649,  554,  353,  347,  339,  260,  243,  215,  181,
        157], dtype=int32)

## 3. Результаты

In [93]:
print(f'i2i:  {round(get_mrr(y, i2i_preds), 4)}')
print(f'IALS: {round(get_mrr(y, ials_preds), 4)}')
print(f'ALS:  {round(get_mrr(y, als_preds[:, :100]),4)}')

i2i:  0.018
IALS: 0.0177
ALS:  0.0124
